In [187]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pyarabic.araby as araby

In [188]:
def get_surah_names():
    surah_names = [] #surah names sorted
    URL = "https://surahquran.com/quran-search/quran.html"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    all_table = soup.find_all('table')[1]
    for idx, elm in enumerate(all_table.find_all("a")):
        surah_names.append(
            (idx + 1, elm.text)
        )
    return surah_names

In [189]:
LANGUAGE = "AR"
file = 'arabic.csv' if LANGUAGE == "AR" else 'french.csv'

book = pd.read_csv(
    f'{os.getcwd()}/data/{file}',
    sep="|",
    header=None,
)

book.columns = ["surah", "ayat", "text"]

book['surah'] = book['surah'].astype(int)
book['ayat'] = book['ayat'].astype(int)
book['text'] = book['text'].apply(lambda t: araby.strip_diacritics(t))

mapping_dictionary = {t[0]: t for t in get_surah_names()}
book['surah_name'] = book['surah'].map(mapping_dictionary).apply(lambda t: t[1])

book['text'].to_csv(f'{os.getcwd()}/output.txt', index=False, sep="\t", header=None)

In [190]:
from thefuzz import fuzz

search_text = "الذين آتيناهم الكتاب يعرفونه كما يعرف"
book = book.assign(
    score = lambda _df: 
        _df['text'].map(lambda t: 
            fuzz.partial_ratio(t, search_text)
        )
    )

In [191]:
book.sort_values('score', ascending=False).loc[0::5, 'text']

0                                  بسم الله الرحمن الرحيم
2238    وتركنا بعضهم يومئذ يموج في بعض  ونفخ في الصور ...
3202    قيل لها ادخلي الصرح  فلما رأته حسبته لجة وكشفت...
1080    وقال الملأ من قوم فرعون أتذر موسى وقومه ليفسدو...
1847                                  ادخلوها بسلام آمنين
                              ...                        
5277    إن ربك هو أعلم بمن ضل عن سبيله وهو أعلم بالمهتدين
5545                                         فرت من قسورة
2590    قل إنما يوحى إلي أنما إلهكم إله واحد  فهل أنتم...
4632                   أإذا متنا وكنا ترابا  ذلك رجع بعيد
5270    قل أرأيتم إن أصبح ماؤكم غورا فمن يأتيكم بماء معين
Name: text, Length: 252, dtype: object